In [1]:
%reload_ext autoreload
%autoreload 2

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
import copy
from pprint import pprint
from pathlib import Path
from typing import Dict, List, Set, Tuple
import spacy
import srsly
# import recon
from recon.corpus import Corpus
from recon.constants import NONE
from recon.corrections import fix_annotations
from recon.dataset import Dataset
from recon.loaders import read_jsonl
from recon.types import Example, PredictionError, HardestExample, NERStats, EntityCoverageStats, EntityCoverage, Transformation, TransformationType
from recon.stats import (
    get_ner_stats, get_entity_coverage, get_sorted_type_counts, counts_to_probs, entropy,
    calculate_entity_coverage_entropy, calculate_label_balance_entropy, calculate_label_distribution_similarity,
    detect_outliers
)
import recon.tokenization as tokenization
from recon.insights import get_ents_by_label, get_label_disparities, top_prediction_errors, top_label_disparities, get_hardest_examples
from recon.recognizer import SpacyEntityRecognizer
from recon.operations import registry
from recon.store import ExampleStore

In [3]:
registry.operations.get_all()

OrderedDict([('fix_tokenization_and_spacing',
              <function recon.operations.operation.__call__.<locals>.factory(dataset, **cfg) -> recon.types.OperationResult>),
             ('add_tokens',
              <function recon.operations.operation.__call__.<locals>.factory(dataset, **cfg) -> recon.types.OperationResult>)])

In [4]:
fix_tokenization_and_spacing = registry.operations.get("fix_tokenization_and_spacing")
add_tokens = registry.operations.get("add_tokens")

In [5]:
corpus = Corpus.from_disk("./data/skills")

In [6]:
corpus.apply_(fix_tokenization_and_spacing)

Running operation on train
Found 3 tokenization errors.
Found 1 unfixable tokenization errors.
TRANSFORMATIONS CALCULATED
Transformations (ADDED):  0
Transformations (REMOVED):  1
Transformations (CHANGED):  1
Running operation on dev
TRANSFORMATIONS CALCULATED
Transformations (ADDED):  0
Transformations (REMOVED):  0
Transformations (CHANGED):  0
Running operation on test
TRANSFORMATIONS CALCULATED
Transformations (ADDED):  0
Transformations (REMOVED):  0
Transformations (CHANGED):  0


In [7]:
corpus.apply_(add_tokens)

Running operation on train
Found 0 tokenization errors.
Found 0 unfixable examples.
TRANSFORMATIONS CALCULATED
Transformations (ADDED):  0
Transformations (REMOVED):  0
Transformations (CHANGED):  76
Running operation on dev
Found 0 tokenization errors.
Found 0 unfixable examples.
TRANSFORMATIONS CALCULATED
Transformations (ADDED):  0
Transformations (REMOVED):  0
Transformations (CHANGED):  61
Running operation on test
Found 0 tokenization errors.
Found 0 unfixable examples.
TRANSFORMATIONS CALCULATED
Transformations (ADDED):  0
Transformations (REMOVED):  0
Transformations (CHANGED):  58


In [8]:
large_corpus = Corpus.from_disk("../../CognitiveServices/API-TextAnalytics-NER.CloudServices/data/2020-03-17/cs/")

In [23]:
large_corpus.apply_(fix_tokenization_and_spacing)

Found 39480 tokenization errors.
Found 1181 unfixable tokenization errors.
Transformations (ADDED):  0
Transformations (REMOVED):  1181
Transformations (CHANGED):  28849
Found 3021 tokenization errors.
Found 79 unfixable tokenization errors.
Transformations (ADDED):  0
Transformations (REMOVED):  79
Transformations (CHANGED):  2232
Found 3219 tokenization errors.
Found 26 unfixable tokenization errors.
Transformations (ADDED):  0
Transformations (REMOVED):  26
Transformations (CHANGED):  2353


In [9]:
large_corpus.apply_(fix_tokenization_and_spacing)

Running operation on train
Found 39480 tokenization errors.
Found 1181 unfixable tokenization errors.
TRANSFORMATIONS CALCULATED
Transformations (ADDED):  0
Transformations (REMOVED):  1181
Transformations (CHANGED):  28849
Running operation on dev
Found 3021 tokenization errors.
Found 79 unfixable tokenization errors.
TRANSFORMATIONS CALCULATED
Transformations (ADDED):  0
Transformations (REMOVED):  79
Transformations (CHANGED):  2232
Running operation on test
Found 3219 tokenization errors.
Found 26 unfixable tokenization errors.
TRANSFORMATIONS CALCULATED
Transformations (ADDED):  0
Transformations (REMOVED):  26
Transformations (CHANGED):  2353


In [ ]:
large_corpus.apply_(add_tokens)

Running operation on train
text='Sukarno' start=19 end=26 label='PERSON' token_start=None token_end=None
text='rabbi' start=97 end=102 label='PERSONTYPE' token_start=None token_end=None
text='2' start=142 end=143 label='NUMBER' token_start=None token_end=None
text='2' start=80 end=81 label='NUMBER' token_start=None token_end=None
text='and a' start=55 end=60 label='PERSONTYPE' token_start=None token_end=None
text='3 November 1913' start=62 end=77 label='DATE' token_start=None token_end=None
text='Anilkumar Muniswamy' start=64 end=83 label='PERSON' token_start=None token_end=None
text='Kanpur' start=105 end=111 label='GPE' token_start=None token_end=None
text='Liberty County' start=65 end=79 label='GPE' token_start=None token_end=None
text='Prabhu Chawla' start=42 end=55 label='PERSON' token_start=None token_end=None
text='1' start=70 end=71 label='NUMBER' token_start=None token_end=None
text='Saxony' start=47 end=53 label='GPE' token_start=None token_end=None
text='disappearance' start

In [3]:
large_corpus_dev = Dataset("dev").from_disk("../../CognitiveServices/API-TextAnalytics-NER.CloudServices/data/2020-03-17/cs/dev.jsonl")

In [ ]:
large_corpus_dev.apply_(fix_tokenization_and_spacing)

In [25]:
2311 - 2232

79

In [44]:
large_corpus_dev = Dataset("dev").from_disk("../../CognitiveServices/API-TextAnalytics-NER.CloudServices/data/2020-03-17/cs/dev.jsonl")
lc_dev_data = large_corpus_dev.data[:1000]
prev_data = copy.deepcopy(lc_dev_data)
prev_data[0]

Example(text='In the days that followed an alleged relationship between Woods and a New York City nightclub hostess sparked a deluge of allegations of additional affairs.', spans=[Span(text='relationship', start=37, end=49, label='EVENT', token_start=None, token_end=None), Span(text='Woods', start=58, end=63, label='PERSON', token_start=None, token_end=None), Span(text='New York City', start=70, end=83, label='GPE', token_start=None, token_end=None), Span(text='nightclub', start=84, end=93, label='LOC', token_start=None, token_end=None), Span(text='hostess', start=94, end=101, label='PERSONTYPE', token_start=None, token_end=None), Span(text='affair', start=148, end=154, label='EVENT', token_start=None, token_end=None)], tokens=[Token(text='In', start=0, end=2, id=0), Token(text='the', start=3, end=6, id=1), Token(text='days', start=7, end=11, id=2), Token(text='that', start=12, end=16, id=3), Token(text='followed', start=17, end=25, id=4), Token(text='an', start=26, end=28, id=5), Toke

In [45]:
new_data = tokenization.fix_tokenization_and_spacing(prev_data)

from timeit import default_timer as timer
print("RUNNING OPERATION\n")

start = timer()

initial_len = len(prev_data)
new_data_len = len(new_data)

def get_examples_and_texts(data: List[Example]) -> Tuple[Dict[int, int], Dict[str, int]]:
    examples: Dict[int, int] = {}
    texts: Dict[str, int] = {}

    for i, e in enumerate(data):
        examples[hash(e)] = i
        texts[e.text_hash()] = i

    return examples, texts

prev_example_hashes, prev_texts = get_examples_and_texts(prev_data)
new_example_hashes, new_texts = get_examples_and_texts(new_data)

prev_diff = set(prev_example_hashes).difference(set(new_example_hashes))
next_diff = set(new_example_hashes).difference(set(prev_example_hashes))

prev_examples_to_hashes = {prev_example_hashes[pd]: pd for pd in prev_diff}
next_examples_to_hashes = {new_example_hashes[nd]: nd for nd in next_diff}

examples_added = max(len(next_diff) - len(prev_diff), 0)
examples_removed = max(len(prev_diff) - len(next_diff), 0)







seen = set()
transformations = set()

for pd in prev_diff:
    if pd in new_example_hashes:
        prev_example_index = prev_example_hashes[pd]
        new_example_hashes[pd] = prev_example_index
    prev_example_index = prev_example_hashes[pd]
    if prev_example_index in next_examples_to_hashes:
        new_example_hash = next_examples_to_hashes[prev_example_index]
        transformation = Transformation(prev_example=pd, example=new_example_hash, type=TransformationType.EXAMPLE_CHANGED)
    else:
        transformation = Transformation(prev_example=pd, example=-1, type=TransformationType.EXAMPLE_REMOVED)
    
    if pd not in seen and new_example_hash not in seen:
        transformations.add(transformation)
        seen.add(pd)
        seen.add(new_example_hash)

# seen = set()

for nd in next_diff:
    next_example_index = new_example_hashes[nd]
    if next_example_index in prev_examples_to_hashes:
        prev_example_hash = prev_examples_to_hashes[next_example_index]
        transformation = Transformation(prev_example=prev_example_hash, example=nd, type=TransformationType.EXAMPLE_CHANGED)
    else:
        transformation = Transformation(prev_example=-1, example=nd, type=TransformationType.EXAMPLE_ADDED)
        
    if nd not in seen and prev_example_hash not in seen:
        transformations.add(transformation)
        seen.add(nd)
        seen.add(prev_example_hash)

        
print(prev_diff)
print(next_diff)
print("Examples Added: ", examples_added)
print("Examples Removed: ", examples_removed)
print("Transformations: ", len(transformations))
print("Transformations (ADDED): ", len([t for t in transformations if t.type == TransformationType.EXAMPLE_ADDED]))
print("Transformations (REMOVED): ", len([t for t in transformations if t.type == TransformationType.EXAMPLE_REMOVED]))
print("Transformations (CHANGED): ", len([t for t in transformations if t.type == TransformationType.EXAMPLE_CHANGED]))



mutual_diff = set(prev_example_hashes) ^ set(new_example_hashes)
end = timer()
print("Total operation calculation time: ", round(end - start, 2))


# from dictdiffer import diff
# start = timer()
# hash_diff = list(diff(prev_data, new_data))
# end = timer()

# print("Total diff of examples: ", round(end - start, 2))

ERROR IN TOKENIZATION:  In the days that followed an alleged relationship between Woods and a New York City nightclub hostess sparked a deluge of allegations of additional affairs. text='affairs' start=148 end=155 label='EVENT' token_start=None token_end=None
ERROR IN TOKENIZATION:  The incident began in downtown Wichita around 4:55 p.m. after "some type of cutting off" occurred, Wichita Police Chief Gordon Ramsay told reporters after the shooting. text='4:55 p.m.' start=46 end=55 label='TIME' token_start=None token_end=None
ERROR IN TOKENIZATION:  The incident began in downtown Wichita around 4:55 p.m. after "some type of cutting off" occurred, Wichita Police Chief Gordon Ramsay told reporters after the shooting. text='reporters' start=139 end=148 label='PERSONTYPE' token_start=None token_end=None
ERROR IN TOKENIZATION:  The city hosts the Neste Oil Rally Finland, which is part of the World Rally Championship. text='World Rally Championship' start=65 end=89 label='EVENT' token_start=N

In [19]:
len(new_data)

9921

In [10]:
len(hash_diff)

9896

In [18]:
print(hash_diff[0][2][0].json())
hash_diff[0][2][1].json()

{"text": "In the days that followed an alleged relationship between Woods and a New York City nightclub hostess sparked a deluge of allegations of additional affairs.", "spans": [{"text": "relationship", "start": 37, "end": 49, "label": "EVENT", "token_start": null, "token_end": null}, {"text": "Woods", "start": 58, "end": 63, "label": "PERSON", "token_start": null, "token_end": null}, {"text": "New York City", "start": 70, "end": 83, "label": "GPE", "token_start": null, "token_end": null}, {"text": "nightclub", "start": 84, "end": 93, "label": "LOC", "token_start": null, "token_end": null}, {"text": "hostess", "start": 94, "end": 101, "label": "PERSONTYPE", "token_start": null, "token_end": null}, {"text": "affair", "start": 148, "end": 154, "label": "EVENT", "token_start": null, "token_end": null}], "tokens": [{"text": "In", "start": 0, "end": 2, "id": 0}, {"text": "the", "start": 3, "end": 6, "id": 1}, {"text": "days", "start": 7, "end": 11, "id": 2}, {"text": "that", "start": 12, "

'{"text": "In the days that followed an alleged relationship between Woods and a New York City nightclub hostess sparked a deluge of allegations of additional affairs.", "spans": [{"text": "relationship", "start": 37, "end": 49, "label": "EVENT", "token_start": null, "token_end": null}, {"text": "Woods", "start": 58, "end": 63, "label": "PERSON", "token_start": null, "token_end": null}, {"text": "New York City", "start": 70, "end": 83, "label": "GPE", "token_start": null, "token_end": null}, {"text": "nightclub", "start": 84, "end": 93, "label": "LOC", "token_start": null, "token_end": null}, {"text": "hostess", "start": 94, "end": 101, "label": "PERSONTYPE", "token_start": null, "token_end": null}, {"text": "affairs", "start": 148, "end": 155, "label": "EVENT", "token_start": null, "token_end": null}], "tokens": [{"text": "In", "start": 0, "end": 2, "id": 0}, {"text": "the", "start": 3, "end": 6, "id": 1}, {"text": "days", "start": 7, "end": 11, "id": 2}, {"text": "that", "start": 12,

In [26]:
len([d for d in hash_diff if d[0] == "remove"])

1

In [41]:
large_corpus.example_store, large_corpus._train.example_store

(<recon.store.ExampleStore at 0x7fe35d856f98>,
 <recon.store.ExampleStore at 0x7fe35d856f98>)

In [32]:
%timeit hash(corpus._train), hash(corpus._dev), hash(corpus._test)

2.85 ms ± 199 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
%timeit hash(large_corpus._train), hash(large_corpus._dev), hash(large_corpus._test)

75.1 ms ± 8.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
%timeit hash(corpus._train)
%timeit hash(large_corpus._train)

872 µs ± 69.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
59.9 ms ± 8.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
len(str(hash(corpus._train.data[0])))

18

In [28]:
hash(corpus._train.data[0].spans[0])

2042131441891384299

In [12]:
corpus._train.commit_hash

'0dd8ebeed75a419cbb1f7e0a9b6244a46e2eebfd'

In [13]:
print(fix_tokenization_and_spacing)

<function operation.__call__.<locals>.factory at 0x7f3f1df8eb70>


In [14]:
corpus._train.operations = []
corpus._train.global_state = {}
corpus._dev.operations = []
corpus._dev.global_state = {}
corpus._test.operations = []
corpus._test.global_state = {}

In [9]:
corpus.apply_(fix_tokenization_and_spacing)

Found 1 tokenization errors.
Found 0 unfixable tokenization errors.
{860580076040132479}
set()
set()


In [20]:
corpus._train.operations

[OperationState(name='fix_tokenization_and_spacing', status=<OperationStatus.COMPLETED: 'COMPLETED'>, ts=datetime.datetime(2020, 4, 7, 13, 4, 42, 400661), examples_added=0, examples_removed=1, examples_corrected=0, annotations_added=0, annotations_removed=0, annotations_corrected=0, transformations=[]),
 OperationState(name='fix_tokenization_and_spacing', status=<OperationStatus.COMPLETED: 'COMPLETED'>, ts=datetime.datetime(2020, 4, 7, 13, 4, 42, 400661), examples_added=0, examples_removed=0, examples_corrected=0, annotations_added=0, annotations_removed=0, annotations_corrected=0, transformations=[]),
 OperationState(name='fix_tokenization_and_spacing', status=<OperationStatus.COMPLETED: 'COMPLETED'>, ts=datetime.datetime(2020, 4, 7, 13, 4, 42, 400661), examples_added=0, examples_removed=0, examples_corrected=0, annotations_added=0, annotations_removed=0, annotations_corrected=0, transformations=[])]

In [21]:
corpus.apply_(add_tokens)

name='add_tokens' status=<OperationStatus.COMPLETED: 'COMPLETED'> ts=datetime.datetime(2020, 4, 7, 13, 4, 42, 400661) examples_added=0 examples_removed=0 examples_corrected=0 annotations_added=0 annotations_removed=0 annotations_corrected=0 transformations=[]
{'fix_tokenization_and_spacing': OperationState(name='fix_tokenization_and_spacing', status=<OperationStatus.COMPLETED: 'COMPLETED'>, ts=datetime.datetime(2020, 4, 7, 13, 4, 42, 400661), examples_added=0, examples_removed=0, examples_corrected=0, annotations_added=0, annotations_removed=0, annotations_corrected=0, transformations=[]), 'add_tokens': OperationState(name='add_tokens', status=<OperationStatus.COMPLETED: 'COMPLETED'>, ts=datetime.datetime(2020, 4, 7, 13, 4, 42, 400661), examples_added=0, examples_removed=0, examples_corrected=0, annotations_added=0, annotations_removed=0, annotations_corrected=0, transformations=[])}
name='add_tokens' status=<OperationStatus.COMPLETED: 'COMPLETED'> ts=datetime.datetime(2020, 4, 7, 13, 

In [22]:
corpus.to_disk('./fixed_data/skills', force=True)

In [141]:
import os
os.listdir('./fixed_data/skills/.recon/train')

['state.json']